In [8]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/medical_insurance.csv")

# One-hot encode categorical variables
df = pd.get_dummies(df, drop_first=True)

# Features and target
X = df.drop("charges", axis=1)
y = df["charges"]

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# ---- Linear Regression (baseline) ----
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)


# ---- Gradient Boosting with GridSearch ----
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
gbr = GradientBoostingRegressor()
gbr_params = {
    "n_estimators": [100, 200],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 4]
}
gbr_grid = GridSearchCV(gbr, gbr_params, cv=3, scoring="r2")
gbr_grid.fit(X_train, y_train)
y_pred_gbr = gbr_grid.best_estimator_.predict(X_test)

# ---- XGBoost with GridSearch ----
from xgboost import XGBRegressor
xgb = XGBRegressor(objective="reg:squarederror", random_state=42)
xgb_params = {
    "n_estimators": [200, 300],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 4]
}
xgb_grid = GridSearchCV(xgb, xgb_params, cv=3, scoring="r2")
xgb_grid.fit(X_train, y_train)
y_pred_xgb = xgb_grid.best_estimator_.predict(X_test)

# ---- AdaBoost with GridSearch ----
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor()
ada_params = {
    "n_estimators": [100, 200],
    "learning_rate": [0.05, 0.1, 0.5]
}
ada_grid = GridSearchCV(ada, ada_params, cv=3, scoring="r2")
ada_grid.fit(X_train, y_train)
y_pred_ada = ada_grid.best_estimator_.predict(X_test)

# ---- LightGBM with GridSearch ----
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor()
lgbm_params = {
    "n_estimators": [100, 200],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 4, -1],
    "num_leaves": [20, 31, 40]
}
lgbm_grid = GridSearchCV(lgbm, lgbm_params, cv=3, scoring="r2")
lgbm_grid.fit(X_train, y_train)
y_pred_lgbm = lgbm_grid.best_estimator_.predict(X_test)

# ---- Evaluation ----
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def metrics(y_true, y_pred):
    return {"R2": r2_score(y_true, y_pred),
            "RMSE": np.sqrt(mean_squared_error(y_true, y_pred))}

# LR
m = metrics(y_test, y_pred_lr)
print("Linear Regression   -> R2:", m["R2"], " RMSE:", m["RMSE"], " Params: None")

# GradientBoosting
m = metrics(y_test, y_pred_gbr)
print("GBR   -> R2:", m["R2"], " RMSE:", m["RMSE"], " Params:", gbr_grid.best_params_)

# XGBoost
m = metrics(y_test, y_pred_xgb)
print("XGB   -> R2:", m["R2"], " RMSE:", m["RMSE"], " Params:", xgb_grid.best_params_)

# AdaBoost
m = metrics(y_test, y_pred_ada)
print("Ada   -> R2:", m["R2"], " RMSE:", m["RMSE"], " Params:", ada_grid.best_params_)

# LightGBM
m = metrics(y_test, y_pred_lgbm)
print("LGBM  -> R2:", m["R2"], " RMSE:", m["RMSE"], " Params:", lgbm_grid.best_params_)





Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [9]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
# ---- Ridge ----
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)
m = metrics(y_test, y_pred_ridge)
print("Ridge     -> R2:", m["R2"], " RMSE:", m["RMSE"], " Params: alpha=1.0")

# ---- Lasso ----
lasso = Lasso(alpha=0.01, max_iter=10000)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
m = metrics(y_test, y_pred_lasso)
print("Lasso     -> R2:", m["R2"], " RMSE:", m["RMSE"], " Params: alpha=0.01")

# ---- ElasticNet ----
enet = ElasticNet(alpha=0.01, l1_ratio=0.5, max_iter=10000)
enet.fit(X_train, y_train)
y_pred_enet = enet.predict(X_test)
m = metrics(y_test, y_pred_enet)
print("ElasticNet-> R2:", m["R2"], " RMSE:", m["RMSE"], " Params: alpha=0.01, l1_ratio=0.5")


Ridge     -> R2: 0.7301966737672894  RMSE: 6061.38994333822  Params: alpha=1.0
Lasso     -> R2: 0.7301408082287657  RMSE: 6062.017447252088  Params: alpha=0.01
ElasticNet-> R2: 0.7301750485245653  RMSE: 6061.632854358161  Params: alpha=0.01, l1_ratio=0.5


In [3]:
pip install lazypredict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.9/705.9 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/7

In [4]:
from lazypredict.Supervised import LazyRegressor

reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

print(models)

  0%|          | 0/42 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000106 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 310
[LightGBM] [Info] Number of data points in the train set: 2217, number of used features: 3
[LightGBM] [Info] Start training from score 13371.074037
                               Adjusted R-Squared  R-Squared       RMSE  \
Model                                                                     
ExtraTreesRegressor                          0.71       0.72    6585.80   
ExtraTreeRegressor                           0.67       0.68    7012.92   
RandomForestRegressor                        0.60       0.61    7759.50   
BaggingRegressor                             0.57       0.58    8026.84   
XGBRegressor                                 0.57       0.58    8067.36   
DecisionTreeRegressor                        0.53       0.53    8464.37   
HistGradientBoostingRegressor                0.34       0

In [10]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.71,0.72,6585.80,0.36
ExtraTreeRegressor,0.67,0.68,7012.92,0.02
RandomForestRegressor,0.60,0.61,7759.50,0.95
BaggingRegressor,0.57,0.58,8026.84,0.10
XGBRegressor,0.57,0.58,8067.36,0.56
DecisionTreeRegressor,0.53,0.53,8464.37,0.02
HistGradientBoostingRegressor,0.34,0.35,9999.00,0.38
LGBMRegressor,0.33,0.34,10067.42,0.08
GradientBoostingRegressor,0.22,0.23,10872.09,0.34
